### LLM Inference Example

This notebook contains a basic inference example for using our `ttml` Python API to build, load, and run a large language model from Hugging Face on our TT hardware. By default, it is set to create and load a GPT2 model, but this notebook can quickly and easily be edited to use any of the LLMs that the tt-train project currently supports. 

Below, in the first cell, we have our imports and basic directory housekeeping.

Use the cell below to change global parameters in this notebook. 

`OUTPUT_TOKENS` : the length of the generated text in token (not characters!) 

`WITH_SAMPLING` : enable or disable output token sampling (only used for PyTorch)

`TEMPERATURE`   : sampling temperature; set to 0 to disable sampling in `generate_with_tt()`

`SEED`          : randomization seed (for reproducibility)

model_id = "meta-llama/Llama-3.2-1B-Instruct" 
CONFIG = "training_shakespeare_llama3_2_1B_fixed.yaml" # now working fine (with proper shape for tokenizer)

model_id =  "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
CONFIG = "training_shakespeare_tinyllama.yaml" # working fine

In [28]:
model_id =  "meta-llama/Llama-3.1-8B-Instruct"
CONFIG = "training_shakespeare_llama3_8B_tp.yaml" # OOM on 12 GB, sucesfully loaded weights

In [29]:
OUTPUT_TOKENS = 100
WITH_SAMPLING = True
TEMPERATURE = 0.8
SEED = 42

import transformers
import torch

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


In [30]:
import os, sys, random
import numpy as np  # For numpy arrays
from dataclasses import dataclass # For configuration classes
from huggingface_hub import hf_hub_download # To download safetensors from Hugging Face
from transformers import AutoTokenizer
from yaml import safe_load # To read YAML configs
from pathlib import Path

import ttml
from ttml.common.config import get_config, TransformerConfig
from ttml.common.utils import set_seed, round_up_to_tile
from ttml.common.model_factory import TransformerModelFactory


In [31]:
# Load the tokenizer from Hugging Face and the transformer config from YAML
tokenizer = AutoTokenizer.from_pretrained(model_id)
transformer_config = TransformerConfig(get_config(CONFIG).get("training_config", {}).get("transformer_config",{}))
yaml_config = get_config(CONFIG)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [34]:
safetensors_path = hf_hub_download(repo_id=model_id, filename="config.json")
safetensors_path = safetensors_path.replace("config.json","")

In [33]:
import torch
from transformers import AutoModelForCausalLM
torch.manual_seed(SEED)
torch_model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16)


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [36]:
tokenizer.vocab_size, torch_model.state_dict()['model.embed_tokens.weight'].shape[0], torch_model.vocab_size

(128000, 128256, 128256)

In [39]:
len(torch_model.state_dict())

291

In [37]:
#total_size = tokenizer.vocab_size + len(tokenizer.added_tokens_decoder)

In [38]:
orig_vocab_size = torch_model.vocab_size
print(orig_vocab_size)
tt_model_factory = TransformerModelFactory(yaml_config)
tt_model_factory.transformer_config.vocab_size = orig_vocab_size

max_sequence_length = tt_model_factory.transformer_config.max_sequence_length

tt_model = tt_model_factory.create_model()
tt_model.load_from_safetensors(safetensors_path)

padded_vocab_size = round_up_to_tile(orig_vocab_size, 32)

if orig_vocab_size != padded_vocab_size:
    print(f"Padding vocab size for tilization: original {orig_vocab_size} -> padded {padded_vocab_size}")


128256
t with shape: Shape([1, 1, 2048, 2048])
Loading tensor: model.layers.8.self_attn.q_proj.weight, shape:Shape([2048, 2048]), dtype:BF16
Using parameter: llama/llama_block_8/attention/q_linear/weight with shape: Shape([1, 1, 2048, 2048])
Loading tensor: model.layers.8.self_attn.v_proj.weight, shape:Shape([512, 2048]), dtype:BF16
Using parameter: llama/llama_block_8/attention/kv_linear/weight with shape: Shape([1, 1, 1024, 2048])
Combined k_proj + v_proj → kv_linear for layer 8
Loading tensor: model.layers.9.input_layernorm.weight, shape:Shape([2048]), dtype:BF16
Using parameter: llama/llama_block_9/attention_norm/gamma with shape: Shape([1, 1, 1, 2048])
Loading tensor: model.layers.9.mlp.down_proj.weight, shape:Shape([2048, 8192]), dtype:BF16
Using parameter: llama/llama_block_9/mlp/w2/weight with shape: Shape([1, 1, 2048, 8192])
Loading tensor: model.layers.9.mlp.gate_proj.weight, shape:Shape([8192, 2048]), dtype:BF16
Using parameter: llama/llama_block_9/mlp/w1/weight with shape: 

RuntimeError: TT_FATAL @ /home/ubuntu/tt-metal/tt_metal/impl/allocator/bank_manager.cpp:431: address.has_value()
info:
Out of Memory: Not enough space to allocate 117440512 B DRAM buffer across 12 banks, where each bank needs to store 9787392 B, but bank size is only 1071181792 B
backtrace:
 --- /home/ubuntu/tt-metal/build/lib/libtt_metal.so(+0x524cdd) [0x7f754d565cdd]
 --- tt::tt_metal::BankManager::allocate_buffer(unsigned long, unsigned long, bool, CoreRangeSet const&, std::optional<unsigned int>, ttsl::StrongType<unsigned int, tt::tt_metal::AllocatorIDTag>)
 --- tt::tt_metal::Allocator::allocate_buffer(tt::tt_metal::Buffer*)
 --- tt::tt_metal::Buffer::allocate_impl()
 --- tt::tt_metal::Buffer::create(tt::tt_metal::IDevice*, unsigned long, unsigned long, tt::tt_metal::BufferType, tt::tt_metal::BufferShardingArgs const&, std::optional<bool>, std::optional<ttsl::StrongType<unsigned char, tt::tt_metal::SubDeviceIdTag> >)
 --- tt::tt_metal::distributed::MeshBuffer::create(std::variant<t

In [ ]:
def build_causal_mask(T: int) -> ttml.autograd.Tensor:
    # [1,1,T,T] float32 with 1s for allowed positions (i >= j), else 0\n",
    m = np.tril(np.ones((T, T), dtype=np.float32))
    return ttml.autograd.Tensor.from_numpy(m.reshape(1, 1, T, T), ttml.Layout.TILE, ttml.autograd.DataType.BFLOAT16)

def build_logits_mask(vocab_size: int, padded_vocab_size: int) -> ttml.autograd.Tensor:
    logits_mask = np.zeros((1, 1, 1, padded_vocab_size), dtype=np.float32)
    logits_mask[:, :, :, vocab_size:] = 1e4
    return ttml.autograd.Tensor.from_numpy(logits_mask, ttml.Layout.TILE, ttml.autograd.DataType.BFLOAT16)   # [1,1,1,T], bfloat16"

`generate_with_tt()` uses TT hardware acceleration to generate output from the chosen LLM

In [ ]:
def generate_with_tt(model, prompt_tokens):
    import time
    
    ttml.autograd.AutoContext.get_instance().set_gradient_mode(ttml.autograd.GradMode.DISABLED)
    model.eval()

    logits_mask_tensor = None

    if padded_vocab_size != orig_vocab_size:
        logits_mask_tensor = build_logits_mask(orig_vocab_size, padded_vocab_size)

    causal_mask = build_causal_mask(max_sequence_length)  # [1,1,seq_len,seq_len], float32
    padded_prompt_tokens = np.zeros((1, 1, 1, max_sequence_length), 
                                    dtype=np.uint32)

    start_idx = 0

    print("************************************")
    start_time = time.time()
    
    for token_idx in range(OUTPUT_TOKENS):

        if len(prompt_tokens) > max_sequence_length:
            start_idx = len(prompt_tokens) - max_sequence_length

        # padded_prompt_tokens[0, 0, 0, :transformer_cfg["max_sequence_length"]] = 0
        padded_prompt_tokens[0, 0, 0, :len(prompt_tokens)] = prompt_tokens[start_idx:]
        padded_prompt_tensor = ttml.autograd.Tensor.from_numpy(
            padded_prompt_tokens,
            ttml.Layout.ROW_MAJOR,
            ttml.autograd.DataType.UINT32)  # [1,1,1, max_seq_len], uint32

        logits = model(padded_prompt_tensor, causal_mask)  # out=[1,1,seq_len, vocab_size], bf16


        next_token_tensor = ttml.ops.sample.sample_op(logits, TEMPERATURE, np.random.randint(low=1e7), logits_mask_tensor)  # out=[1,1,seq_len,1], uint32

        next_token_idx = max_sequence_length - 1 if len(prompt_tokens) > max_sequence_length else len(prompt_tokens) - 1
        next_token = next_token_tensor.to_numpy().flatten()[next_token_idx]

        output = tokenizer.decode(next_token)

        prompt_tokens.append(next_token)
        print(output, end='', flush=True)

    end_time = time.time()
    elapsed_time = end_time - start_time
    tokens_per_second = OUTPUT_TOKENS / elapsed_time
    
    print(f"\n************************************")
    print(f"Generated {OUTPUT_TOKENS} tokens in {elapsed_time:.2f} seconds")
    print(f"Performance: {tokens_per_second:.2f} tokens/second")
    print("************************************\n\n")

In [ ]:
def generate_with_pytorch(torch_model, prompt_tokens):
    import time
    import torch.nn.functional as F
    from transformers import DynamicCache
    
    torch_model.eval()
    
    print("************************************")
    # Convert list to tensor and add batch dimension
    if isinstance(prompt_tokens, list):
        prompt_tokens = torch.tensor([prompt_tokens])
    
    start_time = time.time()
    
    # Initialize KV cache using the new DynamicCache API
    past_key_values = DynamicCache()
    input_ids = prompt_tokens
    
    with torch.no_grad():
        for i in range(OUTPUT_TOKENS):
            # Get model outputs with KV cache
            outputs = torch_model(
                input_ids=input_ids,
                past_key_values=past_key_values,
                use_cache=True
            )
            logits = outputs.logits
            past_key_values = outputs.past_key_values
            
            # Get logits for the last token
            next_token_logits = logits[:, -1, :]
            
            # Apply temperature and sample
            if WITH_SAMPLING and TEMPERATURE > 0:
                next_token_logits = next_token_logits / TEMPERATURE
                probs = F.softmax(next_token_logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)
            else:
                # Greedy sampling
                next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
            
            # Decode and print the token
            output = tokenizer.decode(next_token[0])
            print(output, end='', flush=True)
            
            # For next iteration, only pass the new token (KV cache handles the rest)
            input_ids = next_token
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    tokens_per_second = OUTPUT_TOKENS / elapsed_time
    
    print(f"\n************************************")
    print(f"Generated {OUTPUT_TOKENS} tokens in {elapsed_time:.2f} seconds")
    print(f"Performance: {tokens_per_second:.2f} tokens/second")
    print("************************************\n\n")

In [ ]:
def generate_with_pytorch_batch(torch_model, prompt_tokens):
    """Old version: non-streaming batch generation using torch_model.generate()"""
    import time
    
    torch_model.eval()
    
    print("************************************")
    # Convert list to tensor and add batch dimension
    if isinstance(prompt_tokens, list):
        prompt_tokens = torch.tensor([prompt_tokens])
    
    start_time = time.time()
    
    with torch.no_grad():
        outputs = torch_model.generate(
            prompt_tokens,
            max_new_tokens=OUTPUT_TOKENS,
            do_sample=WITH_SAMPLING,  # Enable sampling
            temperature=TEMPERATURE,   # Temperature for sampling
            num_beams=1  # Use multinomial sampling (standard sampling)
        )
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    tokens_per_second = OUTPUT_TOKENS / elapsed_time
    
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    for t in generated_text:
        print(t)
    
    print(f"\n************************************")
    print(f"Generated {OUTPUT_TOKENS} tokens in {elapsed_time:.2f} seconds")
    print(f"Performance: {tokens_per_second:.2f} tokens/second")
    print("************************************\n\n")


In [ ]:
prompt_str = "Generating with pytorch(CPU, might be slow)"

prompt_tokens = tokenizer.encode(prompt_str)
print("Generating with torch model:")
generate_with_pytorch(torch_model, prompt_tokens)

In [ ]:
prompt_str = "Generating with pytorch(CPU, might be slow)"

prompt_tokens = tokenizer.encode(prompt_str)
print("Generating with torch model batch:")
generate_with_pytorch_batch(torch_model, prompt_tokens)

In [ ]:
prompt_str = "Generating with Tenstorrent, (should be fast, even without kv-caching)"
prompt_tokens = tokenizer.encode(prompt_str)
print("Generating with TT:")
generate_with_tt(tt_model, prompt_tokens.copy())

In [ ]:
def generate_with_pytorch_no_cache(torch_model, prompt_tokens):
    import time
    import torch.nn.functional as F
    
    torch_model.eval()
    
    print("************************************")
    # Convert list to tensor and add batch dimension
    if isinstance(prompt_tokens, list):
        input_ids = torch.tensor([prompt_tokens])
    else:
        input_ids = prompt_tokens
    
    start_time = time.time()
    tokens = 10  # this is very slow
    with torch.no_grad():
        for i in range(tokens):
            # Process the entire sequence every time (no KV cache)
            outputs = torch_model(
                input_ids=input_ids,
                use_cache=False
            )
            logits = outputs.logits
            
            # Get logits for the last token
            next_token_logits = logits[:, -1, :]
            
            # Apply temperature and sample
            if WITH_SAMPLING and TEMPERATURE > 0:
                next_token_logits = next_token_logits / TEMPERATURE
                probs = F.softmax(next_token_logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)
            else:
                # Greedy sampling
                next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
            
            # Decode and print the token
            output = tokenizer.decode(next_token[0])
            print(output, end='', flush=True)
            
            # Append the new token to the full sequence for next iteration
            input_ids = torch.cat([input_ids, next_token], dim=1)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    tokens_per_second = tokens / elapsed_time
    
    print(f"\n************************************")
    print(f"Generated {tokens} tokens in {elapsed_time:.2f} seconds")
    print(f"Performance: {tokens_per_second:.2f} tokens/second")
    print("************************************\n\n")

In [ ]:
prompt_str = "Generating with PyTorch, (should be slow, without kv-caching)"
prompt_tokens = tokenizer.encode(prompt_str)
print("Generating with PyTorch:")
generate_with_pytorch_no_cache(torch_model, prompt_tokens.copy())

In [64]:
sd = torch_model.state_dict()
for s in sd:
    print(s, sd[s].shape)

model.embed_tokens.weight torch.Size([128256, 2048])
model.layers.0.self_attn.q_proj.weight torch.Size([2048, 2048])
model.layers.0.self_attn.k_proj.weight torch.Size([512, 2048])
model.layers.0.self_attn.v_proj.weight torch.Size([512, 2048])
model.layers.0.self_attn.o_proj.weight torch.Size([2048, 2048])
model.layers.0.mlp.gate_proj.weight torch.Size([8192, 2048])
model.layers.0.mlp.up_proj.weight torch.Size([8192, 2048])
model.layers.0.mlp.down_proj.weight torch.Size([2048, 8192])
model.layers.0.input_layernorm.weight torch.Size([2048])
model.layers.0.post_attention_layernorm.weight torch.Size([2048])
model.layers.1.self_attn.q_proj.weight torch.Size([2048, 2048])
model.layers.1.self_attn.k_proj.weight torch.Size([512, 2048])
model.layers.1.self_attn.v_proj.weight torch.Size([512, 2048])
model.layers.1.self_attn.o_proj.weight torch.Size([2048, 2048])
model.layers.1.mlp.gate_proj.weight torch.Size([8192, 2048])
model.layers.1.mlp.up_proj.weight torch.Size([8192, 2048])
model.layers.1.

In [65]:
k = tt_model.parameters()
for s in k:
    print(s, k[s].shape())

llama/llama_block_9/mlp/w3/weight [1, 1, 8192, 2048]
llama/llama_block_9/mlp/w1/weight [1, 1, 8192, 2048]
llama/llama_block_1/mlp/w1/weight [1, 1, 8192, 2048]
llama/llama_block_5/mlp/w1/weight [1, 1, 8192, 2048]
llama/llama_block_11/attention/kv_linear/weight [1, 1, 1024, 2048]
llama/llama_block_10/attention/out_linear/weight [1, 1, 2048, 2048]
llama/llama_block_8/attention_norm/gamma [1, 1, 1, 2048]
llama/llama_block_2/mlp/w3/weight [1, 1, 8192, 2048]
llama/llama_block_10/attention/kv_linear/weight [1, 1, 1024, 2048]
llama/llama_block_10/mlp/w3/weight [1, 1, 8192, 2048]
llama/llama_block_10/mlp_norm/gamma [1, 1, 1, 2048]
llama/llama_block_2/attention_norm/gamma [1, 1, 1, 2048]
llama/llama_block_1/mlp/w3/weight [1, 1, 8192, 2048]
llama/llama_block_1/mlp/w2/weight [1, 1, 2048, 8192]
llama/llama_block_1/attention/q_linear/weight [1, 1, 2048, 2048]
llama/llama_block_1/attention/out_linear/weight [1, 1, 2048, 2048]
llama/llama_block_0/mlp/w2/weight [1, 1, 2048, 8192]
llama/llama_block_12/a

In [66]:
import numpy as np
import torch


def apply_rope_permutation(w, num_heads):
    """
    Apply RoPE row permutation to match TT's q_proj weight layout.
    TT applies unpermute_proj_rows during loading which interleaves rows within each head.
    """
    rows, cols = w.shape
    head_dim = rows // num_heads
    
    out = np.zeros_like(w)
    for h in range(num_heads):
        head_start = h * head_dim
        half = head_dim // 2
        
        # Interleave: [0..half-1, half..head_dim-1] → [0, half, 1, half+1, ..., half-1, head_dim-1]
        for i in range(half):
            out[head_start + 2*i] = w[head_start + i]
            out[head_start + 2*i + 1] = w[head_start + half + i]
    
    return out


def compare_weights(torch_model, tt_model):
    """
    Compare weights between PyTorch model and TT-Metal model.
    
    Args:
        torch_model: PyTorch model (with .state_dict())
        tt_model: TT-Metal model (with .parameters())
    """
    
    torch_sd = torch_model.state_dict()
    tt_params = tt_model.parameters()
    
    # Get num_heads from torch model config
    num_heads = torch_model.config.num_attention_heads
    
    # Detect weight tying configuration from TT model
    has_tok_emb = 'llama/tok_emb/weight' in tt_params
    has_fc = 'llama/fc/weight' in tt_params
    weight_tying_enabled = not has_tok_emb  # If tok_emb doesn't exist, weight tying is enabled
    
    # Mapping from PyTorch parameter names to TT-Metal parameter names
    pytorch_to_tt_mapping = {
        # Final layer norm
        'model.norm.weight': 'llama/ln_fc/gamma',
    }
    
    # Add embedding mappings based on weight tying configuration
    if weight_tying_enabled:
        # Weight tying enabled: both embed_tokens and lm_head use fc/weight
        pytorch_to_tt_mapping['model.embed_tokens.weight'] = 'llama/fc/weight'
        # lm_head.weight should also map to fc/weight, but we'll skip it to avoid duplicate checks
    else:
        # Weight tying disabled: separate tok_emb and fc
        pytorch_to_tt_mapping['model.embed_tokens.weight'] = 'llama/tok_emb/weight'
        pytorch_to_tt_mapping['lm_head.weight'] = 'llama/fc/weight'
    
    # Add layer-specific mappings
    for i in range(50):  # Support up to 50 layers
        layer_prefix_pt = f'model.layers.{i}'
        layer_prefix_tt = f'llama/llama_block_{i}'
        
        pytorch_to_tt_mapping.update({
            f'{layer_prefix_pt}.input_layernorm.weight': f'{layer_prefix_tt}/attention_norm/gamma',
            f'{layer_prefix_pt}.post_attention_layernorm.weight': f'{layer_prefix_tt}/mlp_norm/gamma',
            f'{layer_prefix_pt}.self_attn.q_proj.weight': f'{layer_prefix_tt}/attention/q_linear/weight',
            f'{layer_prefix_pt}.self_attn.o_proj.weight': f'{layer_prefix_tt}/attention/out_linear/weight',
            # k_proj and v_proj are combined into kv_linear in TT
            f'{layer_prefix_pt}.mlp.gate_proj.weight': f'{layer_prefix_tt}/mlp/w1/weight',
            f'{layer_prefix_pt}.mlp.up_proj.weight': f'{layer_prefix_tt}/mlp/w3/weight',
            f'{layer_prefix_pt}.mlp.down_proj.weight': f'{layer_prefix_tt}/mlp/w2/weight',
        })
    
    print("=" * 80)
    print("WEIGHT COMPARISON: PyTorch vs TT-Metal")
    print("=" * 80)
    print(f"Note: Detected num_heads={num_heads} for RoPE permutation")
    print(f"Note: Weight tying {'ENABLED' if weight_tying_enabled else 'DISABLED'}")
    print("=" * 80)
    
    mismatches = []
    matches = []
    
    for pt_name in torch_sd.keys():
        if 'bias' in pt_name:
            continue  # Skip bias parameters
            
        pt_tensor = torch_sd[pt_name]
        pt_shape = tuple(pt_tensor.shape)
        
        # Handle k_proj and v_proj specially (they're combined in TT)
        if '.self_attn.k_proj.weight' in pt_name or '.self_attn.v_proj.weight' in pt_name:
            layer_idx = pt_name.split('.')[2]
            tt_name = f'llama/llama_block_{layer_idx}/attention/kv_linear/weight'
            
            if tt_name in tt_params:
                tt_tensor_np = tt_params[tt_name].to_numpy()
                tt_shape = tt_tensor_np.shape
                
                # k_proj and v_proj are concatenated in kv_linear
                # Expected: k_proj [512, 2048] + v_proj [512, 2048] = kv_linear [1, 1, 1024, 2048]
                if '.self_attn.k_proj.weight' in pt_name:
                    print(f"\n{pt_name}")
                    print(f"  PyTorch: {pt_shape}")
                    print(f"  TT (kv combined): {tt_shape}")
                    print(f"  Status: K and V are combined in TT as kv_linear")
            continue
        
        # Get corresponding TT parameter name
        tt_name = pytorch_to_tt_mapping.get(pt_name)
        if not tt_name:
            continue
            
        if tt_name not in tt_params:
            print(f"\n❌ MISSING: {pt_name} -> {tt_name}")
            print(f"   PyTorch shape: {pt_shape}")
            mismatches.append((pt_name, "MISSING IN TT"))
            continue
        
        # Get TT tensor
        tt_tensor_np = tt_params[tt_name].to_numpy()
        tt_shape = tt_tensor_np.shape
        
        # Remove batch dimensions [1, 1, ...] from TT tensor
        tt_shape_no_batch = tt_shape[2:] if len(tt_shape) == 4 else tt_shape
        
        # Compare shapes
        pt_numpy = pt_tensor.cpu().float().numpy()  # Convert to float32 for numpy compatibility
        
        # Special handling for q_proj: TT applies RoPE row permutation during loading
        is_q_proj = '.self_attn.q_proj.weight' in pt_name
        if is_q_proj and len(pt_shape) == 2 and pt_shape[0] % num_heads == 0:
            pt_numpy = apply_rope_permutation(pt_numpy, num_heads)
        
        # For layer norms: PT (N,) vs TT (1, N) - both are fine, just broadcasting
        # Check if PT is 1D and TT has leading 1s that can be squeezed
        if len(pt_shape) == 1 and len(tt_shape_no_batch) == 2 and tt_shape_no_batch[0] == 1:
            tt_shape_no_batch = (tt_shape_no_batch[1],)  # Squeeze leading 1
        
        # Check if shapes match (with or without transpose)
        shape_match = (pt_shape == tt_shape_no_batch) or (pt_shape == tt_shape_no_batch[::-1])
        
        if shape_match:
            # Check actual values
            # Reshape TT data to match PT shape (handle batch dims and potential squeezing)
            if len(tt_shape) == 4:
                tt_data = tt_tensor_np.reshape(tt_shape[2:])  # Remove [1,1,...] batch dims
            else:
                tt_data = tt_tensor_np.reshape(tt_shape)
            
            # Squeeze if needed for 1D comparisons
            tt_data = tt_data.squeeze()
            pt_numpy_squeezed = pt_numpy.squeeze()
            
            # Handle transpose if needed
            if pt_numpy_squeezed.shape != tt_data.shape and len(tt_data.shape) == 2:
                tt_data = tt_data.T
            
            diff = np.abs(pt_numpy_squeezed - tt_data).max()
            rel_diff = diff / (np.abs(pt_numpy_squeezed).max() + 1e-8)
            
            status = "✓" if diff < 1e-3 else "⚠"
            note = " (after RoPE permutation)" if is_q_proj else ""
            print(f"\n{status} {pt_name}{note}")
            print(f"  PyTorch: {pt_shape}")
            print(f"  TT:      {tt_shape} -> {tt_shape_no_batch}")
            print(f"  Max diff: {diff:.6f}, Rel diff: {rel_diff:.6f}")
            
            if diff < 1e-3:
                matches.append(pt_name)
            else:
                mismatches.append((pt_name, f"VALUE_DIFF={diff:.6f}"))
        else:
            print(f"\n❌ SHAPE MISMATCH: {pt_name}")
            print(f"  PyTorch: {pt_shape}")
            print(f"  TT:      {tt_shape} -> {tt_shape_no_batch}")
            mismatches.append((pt_name, f"SHAPE: PT={pt_shape} vs TT={tt_shape_no_batch}"))
    
    print("\n" + "=" * 80)
    print(f"SUMMARY: {len(matches)} matches, {len(mismatches)} mismatches")
    print("=" * 80)
    
    if mismatches:
        print("\n❌ MISMATCHES:")
        for name, issue in mismatches:
            print(f"  - {name}: {issue}")
    
    return matches, mismatches


# Usage example (commented out):
matches, mismatches = compare_weights(torch_model, tt_model)



WEIGHT COMPARISON: PyTorch vs TT-Metal
Note: Detected num_heads=32 for RoPE permutation
Note: Weight tying ENABLED

✓ model.embed_tokens.weight
  PyTorch: (128256, 2048)
  TT:      (1, 1, 128256, 2048) -> (128256, 2048)
  Max diff: 0.000000, Rel diff: 0.000000

✓ model.layers.0.self_attn.q_proj.weight (after RoPE permutation)
  PyTorch: (2048, 2048)
  TT:      (1, 1, 2048, 2048) -> (2048, 2048)
  Max diff: 0.000000, Rel diff: 0.000000

model.layers.0.self_attn.k_proj.weight
  PyTorch: (512, 2048)
  TT (kv combined): (1, 1, 1024, 2048)
  Status: K and V are combined in TT as kv_linear

✓ model.layers.0.self_attn.o_proj.weight
  PyTorch: (2048, 2048)
  TT:      (1, 1, 2048, 2048) -> (2048, 2048)
  Max diff: 0.000000, Rel diff: 0.000000

✓ model.layers.0.mlp.gate_proj.weight
  PyTorch: (8192, 2048)
  TT:      (1, 1, 8192, 2048) -> (8192, 2048)
  Max diff: 0.000000, Rel diff: 0.000000

✓ model.layers.0.mlp.up_proj.weight
  PyTorch: (8192, 2048)
  TT:      (1, 1, 8192, 2048) -> (8192, 2048)


✓ model.layers.6.mlp.up_proj.weight
  PyTorch: (8192, 2048)
  TT:      (1, 1, 8192, 2048) -> (8192, 2048)
  Max diff: 0.000000, Rel diff: 0.000000

✓ model.layers.6.mlp.down_proj.weight
  PyTorch: (2048, 8192)
  TT:      (1, 1, 2048, 8192) -> (2048, 8192)
  Max diff: 0.000000, Rel diff: 0.000000

✓ model.layers.6.input_layernorm.weight
  PyTorch: (2048,)
  TT:      (1, 1, 1, 2048) -> (2048,)
  Max diff: 0.000000, Rel diff: 0.000000

✓ model.layers.6.post_attention_layernorm.weight
  PyTorch: (2048,)
  TT:      (1, 1, 1, 2048) -> (2048,)
  Max diff: 0.000000, Rel diff: 0.000000

✓ model.layers.7.self_attn.q_proj.weight (after RoPE permutation)
  PyTorch: (2048, 2048)
  TT:      (1, 1, 2048, 2048) -> (2048, 2048)
  Max diff: 0.000000, Rel diff: 0.000000

model.layers.7.self_attn.k_proj.weight
  PyTorch: (512, 2048)
  TT (kv combined): (1, 1, 1024, 2048)
  Status: K and V are combined in TT as kv_linear

✓ model.layers.7.self_attn.o_proj.weight
  PyTorch: (2048, 2048)
  TT:      (1, 1, 2


✓ model.layers.13.mlp.up_proj.weight
  PyTorch: (8192, 2048)
  TT:      (1, 1, 8192, 2048) -> (8192, 2048)
  Max diff: 0.000000, Rel diff: 0.000000

✓ model.layers.13.mlp.down_proj.weight
  PyTorch: (2048, 8192)
  TT:      (1, 1, 2048, 8192) -> (2048, 8192)
  Max diff: 0.000000, Rel diff: 0.000000

✓ model.layers.13.input_layernorm.weight
  PyTorch: (2048,)
  TT:      (1, 1, 1, 2048) -> (2048,)
  Max diff: 0.000000, Rel diff: 0.000000

✓ model.layers.13.post_attention_layernorm.weight
  PyTorch: (2048,)
  TT:      (1, 1, 1, 2048) -> (2048,)
  Max diff: 0.000000, Rel diff: 0.000000

✓ model.layers.14.self_attn.q_proj.weight (after RoPE permutation)
  PyTorch: (2048, 2048)
  TT:      (1, 1, 2048, 2048) -> (2048, 2048)
  Max diff: 0.000000, Rel diff: 0.000000

model.layers.14.self_attn.k_proj.weight
  PyTorch: (512, 2048)
  TT (kv combined): (1, 1, 1024, 2048)
  Status: K and V are combined in TT as kv_linear

✓ model.layers.14.self_attn.o_proj.weight
  PyTorch: (2048, 2048)
  TT:      (